# Generate one year of demand based on calendar

In [71]:
import holidays
import yaml

from copy import deepcopy
from datetime import date, timedelta

In [72]:
with open('../configs/renfe_tft/demand_data_base.yml', 'r') as f:
    demand_yaml = f.read()

data = yaml.safe_load(demand_yaml)

In [73]:
class Calendar1(holidays.HolidayBase):

    def _populate(self, year):
        # Febraury
        for i in range(1, 29):
            self[date(year, 2, i)] = 'Holidays'
        # March - July - October
        for i in range(1, 32):
            self[date(year, 3, i)] = 'Holidays'
            self[date(year, 7, i)] = 'Holidays'
            self[date(year, 10, i)] = 'Holidays'
        # June - November
        for i in range(1, 31):
            self[date(year, 6, i)] = 'Holidays'
            self[date(year, 11, i)] = 'Holidays'

In [74]:
class Calendar2(holidays.HolidayBase):

    def _populate(self, year):
        # January - May - August - December
        for i in range(1, 32):
            self[date(year, 1, i)] = 'Holidays'
            self[date(year, 5, i)] = 'Holidays'
            self[date(year, 8, i)] = 'Holidays'
            self[date(year, 12, i)] = 'Holidays'
        # April - September
        for i in range(1, 31):
            self[date(year, 4, i)] = 'Holidays'
            self[date(year, 9, i)] = 'Holidays'

In [75]:
class CustomHolidays(holidays.ES):
    
    def _populate(self, year):
        holidays.ES._populate(self, year)
        for i in range(1, 7):
            self[date(year, 1, i)] = 'Navidad'
        self[date(year, 2, 14)] = 'San Laventín'
        for i in range(8, 14):
            self[date(year, 3, i)] = 'Carnaval'
        for i in range(2, 10):
            self[date(year, 4, i)] = 'Semana Santa'
        self[date(year, 8, 1)] = 'Verano'
        self[date(year, 8, 31)] = 'Verano'
        for i in range(24, 26):
            self[date(year, 12, i)] = 'Navidad'

In [76]:
custom_holidays = holidays.ES()
calendar1 = Calendar1()
calendar2 = Calendar2()

def get_demand_pattern(_date: date):
    demand_pattern = None
    if _date.weekday() >= 0 and _date.weekday() <= 3:
        if _date in calendar1:
            demand_pattern = 1 # Monday-Thrusday-1
        elif _date in calendar2:
            demand_pattern = 3 # Monday-Thrusday-2
    elif _date.weekday() > 3 and _date.weekday() <= 6:
        if _date in calendar1:
            demand_pattern = 2 # Weekend-1
        elif _date in calendar2:
            demand_pattern = 4 # Weekend-2
    if _date in custom_holidays:
        demand_pattern = 4 # Weekend-2
    return demand_pattern

In [77]:
start_date = date(2023, 1, 1)
end_date = date(2023, 12, 31)
while start_date < end_date:
    previous_day = deepcopy(data['day'][-1])
    previous_day['id'] += 1
    previous_day['date'] += timedelta(days=1)
    previous_day['demandPattern'] = get_demand_pattern(previous_day['date'])
    data['day'].append(previous_day)
    start_date += timedelta(days=1)

In [78]:
yaml.safe_dump(data, open('../configs/renfe_tft/demand_data.yml', 'w'), sort_keys=False, allow_unicode=True)